# Toy Radix Benders Multicut

In [1]:
SCALE_CROWDING=False
PREVENT_ZERO = True
REG_WEIGHT = 1e-5    # Regularization weight

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f2db240f6c8>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
mdl_ref = ijomc
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound = crowding_bound0

not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

#N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

## Need to allow higher growth for acetate with higher uptake rate

In [7]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

In [8]:
from dynamicme.decomposition import BendersSubmodel, BendersMaster
from dynamicme.generate import copy_model

In [9]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = (1./REG_WEIGHT) #1.
    sn.objective_coefficient = (1./REG_WEIGHT) #1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------
    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:    
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = BendersSubmodel(gap, cond)
    sub_dict[cond] = sub
    
for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = REG_WEIGHT
    
master = BendersMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter Presolve to 0
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


In [10]:
master.add_submodels(sub_dict)

In [11]:
zcut = master.model.getConstrByName('z_cut')
print(master.model.getRow(zcut), zcut.Sense, zcut.RHS)

(<gurobi.LinExpr: z + -1e-05 binary_G6PDH2r00 + -1e-05 binary_G6PDH2r02 + -1e-05 binary_AKGDH00 + -1e-05 binary_AKGDH02 + -1e-05 binary_ME200 + -1e-05 binary_ME202 + -1e-05 binary_PGK00 + -1e-05 binary_PGK02 + -1e-05 binary_GLUN00 + -1e-05 binary_GLUN02 + -1e-05 binary_ME100 + -1e-05 binary_ME102 + -1e-05 binary_GND00 + -1e-05 binary_GND02 + -1e-05 binary_ACKr00 + -1e-05 binary_ACKr02 + -1e-05 binary_GLNS00 + -1e-05 binary_GLNS02 + -1e-05 binary_ADK100 + -1e-05 binary_ADK102 + -1e-05 binary_SUCDi00 + -1e-05 binary_SUCDi02 + -1e-05 binary_PPC00 + -1e-05 binary_PPC02 + -1e-05 binary_MDH00 + -1e-05 binary_MDH02 + -1e-05 binary_FUM00 + -1e-05 binary_FUM02 + -1e-05 binary_GLUDy00 + -1e-05 binary_GLUDy02 + -1e-05 binary_GLUSy00 + -1e-05 binary_GLUSy02 + -1e-05 binary_PGL00 + -1e-05 binary_PGL02 + -1e-05 binary_PGM00 + -1e-05 binary_PGM02 + -1e-05 binary_ACALD00 + -1e-05 binary_ACALD02 + -1e-05 binary_MALS00 + -1e-05 binary_MALS02 + -1e-05 binary_PGI00 + -1e-05 binary_PGI02 + -1e-05 binary_PP

In [12]:
from dynamicme.callback_gurobi import cb_benders_multi

In [13]:
master.model._precision_sub = 'gurobi'
master.model._verbosity = 0
#master.model.Params.MIPGap = 0.001
master.model.Params.OutputFlag = 1
# master.model.Params.TimeLimit = 60
master.model.optimize(cb_benders_multi)

Parameter OutputFlag unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 193 rows, 147 columns and 387 nonzeros
Variable types: 3 continuous, 144 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-05, 1e+00]
  Objective range  [1e-05, 1e-05]
  Bounds range     [1e+00, 1e+03]
  RHS range        [9e-01, 1e+00]
Found heuristic solution: objective 0.0003700
Variable types: 3 continuous, 144 integer (144 binary)

Root relaxation: objective 0.000000e+00, 48 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       0.0000000    0.00000  0.00%     -    0s

Explored 0 nodes (48 simplex iterations) in 0.02 seconds
Thread count was 8 (of 8 available processors)

Solution count 2: 0 0.00037 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%

Exception TypeError: "optimize() got an unexpected keyword argument 'precision'" in 'gurobipy.callbackstub' ignored
Exception TypeError: "optimize() got an unexpected keyword argument 'precision'" in 'gurobipy.callbackstub' ignored


In [14]:
sol_master = {x.VarName:x.X for x in master.model.getVars()}

In [15]:
yopt = [sol_master[y.VarName] for y in master._ys]
print('Number of non-zero binaries: %g' % sum(yopt))
# Fitted parameters

kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

Number of non-zero binaries: 48


In [16]:
kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
kfit_changed

Changed keffs: 0/48


[]

In [17]:
#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')    
    crowding = Constraint('crowding')
    crowding._bound = crowding_bound0
    crowding._constraint_sense = 'L'
    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit})
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Cond=glucose. mu_meas=0.74. mu_sim=0.873922 (unfit=0.873922, error=18.1%). Error=18.1% (0% change)
	EX_glc__D_e uptake=-10
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.389313 (unfit=0.389313, error=52.1%). Error=52.1% (0% change)
	EX_glc__D_e uptake=0
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=-20


dual_sp = master.sub_dict['acetate'].model.getConstrByName('sp_acetate')
dual_sn = master.sub_dict['acetate'].model.getConstrByName('sn_acetate')

sub = master.sub_dict['acetate']

print(sub.model.model.getRow(dual_sp), dual_sp.Sense, dual_sp.RHS)
print(sub.model.model.getRow(dual_sn), dual_sn.Sense, dual_sn.RHS)

master.sub_dict['acetate'].model.getConstrByName(mu_id+'_acetate')

[(c, c.RHS) for c in sub.model.getConstrs() if c.RHS != 0]

[(rxn.id, rxn.objective_coefficient) for rxn in sub.cobra_model.reactions if rxn.objective_coefficient != 0]

tk = master.model.getVarByName('tk_acetate')
wa = np.array([sub.model.x_dict[w.VarName] for w in sub._wa])
wl = np.array([sub.model.x_dict[w.VarName] for w in sub._wl])
wu = np.array([sub.model.x_dict[w.VarName] for w in sub._wu])
d = sub._d
Bcsc = sub._Bcsc
waB = wa*Bcsc
cinds = waB.nonzero()[0]
ys = master._ys
xl = sub._xl
xu = sub._xu
#waByopt = LinExpr([waB[j] for j in cinds], [ys[j] for j in cinds])
waByopt = sum(waB * yopt)
sum(d*wa) -waByopt + sum(xl*wl) - sum(xu*wu)

tk.X